In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
data

In [ ]:
(data['Attrition'].value_counts()/1470)*100

In [ ]:
data['BusinessTravel'].value_counts().plot(kind='barh',color=['r','g','b'])

In [ ]:
data['Department'].value_counts().plot(kind='barh',color=['r','g','b'])

In [ ]:
data['EducationField'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])

In [ ]:
plt.hist(data['DistanceFromHome'])
plt.xlabel('Distance From Home')

In [ ]:
plt.hist(data['Education'])
plt.xlabel('Education')

In [ ]:
plt.hist(data['DailyRate'])
plt.xlabel('DailyRate')

In [ ]:
plt.hist(data['Attrition'])

In [ ]:
data[data['Attrition']=='Yes']['BusinessTravel'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('Business Travel | Attrition=Yes')

In [ ]:
data[data['Attrition']=='No']['BusinessTravel'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('Business Travel | Attrition=No')

In [ ]:
data[data['Attrition']=='Yes']['Department'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('Department | Attrition=Yes')

In [ ]:
data[data['Attrition']=='No']['Department'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('Department | Attrition=No')

In [ ]:
data[data['Attrition']=='Yes']['EducationField'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('EducationField | Attrition=Yes')

In [ ]:
data[data['Attrition']=='No']['EducationField'].value_counts().plot(kind='barh',color=['r','g','b','y','c','#000456'])
plt.xlabel('EducationField | Attrition=No')

In [ ]:
data['Attrition'].replace({'Yes':1,'No':0},inplace=True)

In [ ]:
data.info()

In [ ]:
plt.scatter(data['Gender'],data['TotalWorkingYears'],c=data['Attrition'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
l1=LabelEncoder()
features=[]
for ft in data:
    if data[ft].dtype=='O':
        features.append(ft)

In [ ]:
for f in features:
    data[f]=l1.fit_transform(data[f])

In [ ]:
data

In [ ]:
data.corr()

In [ ]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
X=data.drop(['Attrition','EmployeeCount','StandardHours','Over18'],axis=1)
y=data['Attrition']

In [ ]:
X_os,y_os=sm.fit_resample(X,y)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
Xos=sc.fit_transform(X_os)

In [ ]:
X=sc.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
Xos_train,Xos_test,Yos_train,Yos_test=train_test_split(Xos,y_os)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y)

In [ ]:
model_test_os=[]
model_train_os=[]
model_test=[]
model_train=[]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC  

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
models=[RandomForestClassifier(),LogisticRegression(),AdaBoostClassifier(),DecisionTreeClassifier(),SVC()]
for md in models:
    md_os=md
    md_os.fit(Xos_train,Yos_train)
    y_preos=md_os.predict(Xos_test)
    md_s=md
    md_s.fit(X_train,Y_train)
    y_pre=md_s.predict(X_test)
    model_test_os.append(f1_score(y_preos,Yos_test))
    model_train_os.append(md_os.score(Xos_train,Yos_train))
    model_test.append(f1_score(y_pre,Y_test))
    model_train.append(md_s.score(X_train,Y_train))

In [ ]:
model_test_os

In [ ]:
model_test

In [ ]:
plt.plot(model_test_os,color='blue',label='F1 Score Over Sampled Test Data')
plt.plot(model_test,color='red',label='F1 Score Biased Test data')
plt.legend()
plt.plot()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(15,7.5)})

In [ ]:
df1=[]
df1=pd.DataFrame(df1)

In [ ]:
df1['models']=['Random Forest','Logistic Regression','Adaboost','Decision Trees','Support Vector Machines']

In [ ]:
df1['OS Acc']=model_test_os

In [ ]:
df1

In [ ]:
df2=[]
df2=pd.DataFrame(df2)

In [ ]:
df2['models']=['Random Forest','Logistic Regression','Adaboost','Decision Trees','Support Vector Machines']
df2['Acc']=model_test

In [ ]:
sns.lineplot(data=df1,x='models',y='OS Acc',label='F1 Score Over Sampled Test Data')
sns.lineplot(data=df2,x='models',y='Acc',label='F1 Score Biased Test Data')
plt.legend()

In [ ]:
sns.barplot(x=models,y=model_test_os,saturation=0.5)
sns.barplot(x=models,y=model_test)